In [1]:
!pip install geopy

In [2]:
import numpy as np
import pandas as pd
import math as m
import matplotlib.pyplot as plt
import seaborn as sns
import json
from geopy.distance import geodesic
import folium

# Indicateur : Nombre d'accidents à vélo

## Base de données accidents

In [ ]:
#Nom dossier : 'BaseDeDonnées/Accidents_france/caract-2023.csv'
# Nom dossier : 'BaseDeDonnées/Accidents_france/caract-2022.csv' à rajouter suivant le notebook utilisé google colab ou jupyter
df_accidents_2023 = pd.read_csv("caract-2023.csv",sep=';') #2023
df_accidents_2022 = pd.read_csv("carcteristiques-2022.csv",sep=';')#2022

In [ ]:

df_accidents_2022 = df_accidents_2022.rename(columns={'Accident_Id':'Num_Acc'})

In [ ]:
#df_vehicules = pd.read_csv('BaseDeDonnées/Accidents_france/vehicles.csv')
# Nom dossier : 'BaseDeDonnées/Accidents_france/vehicules-2023.csv'
#Nom dossier : 'BaseDeDonnées/Accidents_france/vehicules-2022.csv'
df_vehicules_2023 = pd.read_csv('vehicules-2023.csv',sep=';')
df_vehicules_2022 = pd.read_csv('vehicules-2022.csv',sep=';')

## Filtre par département et zone urbaine

In [ ]:
df_accidents_grenoble_2023 = df_accidents_2023[df_accidents_2023['dep']=='38'] # 2023
df_accidents_grenoble_2022 = df_accidents_2022[(df_accidents_2022['dep']== '38')] #2022

## Filtre sur les véhicules qui sont des vélos

In [ ]:
df_vehicules_2023 = df_vehicules_2023[df_vehicules_2023['catv']==1]
df_vehicules_2022 = df_vehicules_2022[df_vehicules_2022['catv']==1]

## Merge avec Vehicules

In [ ]:
df_accidents_grenoble_2023 = pd.merge(df_accidents_grenoble_2023,df_vehicules_2023, how='inner',on='Num_Acc')
df_accidents_grenoble_2022 = pd.merge(df_accidents_grenoble_2022,df_vehicules_2022, how='inner',on='Num_Acc')


In [ ]:
df_accidents_grenoble_2023 = df_accidents_grenoble_2023[df_accidents_grenoble_2023['catv']==1]
df_accidents_grenoble_2022 = df_accidents_grenoble_2022[df_accidents_grenoble_2022['catv']==1]


In [ ]:
print('Le nombre d\'accidents en 2023 était de : {} et de {} en 2022'.format(len(df_accidents_grenoble_2023),len(df_accidents_grenoble_2022)))

Le nombre d'accidents en 2023 était de : 68 et de 68 en 2022


L'étude de ces fichiers n'est pas terminée mais déjà on peut avoir une idée du nombre d'accidents impliquant des vélos et voir si l'on peut en tirer quelque chose.
Entre 2005 - 2016  , il y a eu 448 accidents de vélo recensés par la police à Grenoble.

## Map des accidents qu'il y eu impliquant un cycliste dans l'agglomération Grenobloise.

In [ ]:
df_accidents_grenoble_2023['lat']= df_accidents_grenoble_2023['lat'].str.replace(',','.')
df_accidents_grenoble_2023['long']= df_accidents_grenoble_2023['long'].str.replace(',','.')

In [ ]:
map_accidents_carrefour_Grenoble = folium.Map(location=[45.188529, 5.724524],
                                              zoom_start=14,
                                              titles = 'Map Accidents Carrefour Grenoble')


In [ ]:
df_accidents_grenoble_2023['lat']=df_accidents_grenoble_2023['lat'].astype(object)
df_accidents_grenoble_2023['long']=df_accidents_grenoble_2023['long'].astype(object)

for index, row in df_accidents_grenoble_2023.iterrows():
    folium.Marker([row['lat'], row['long']],icon=folium.Icon(color="black")).add_to(map_accidents_carrefour_Grenoble)

## Comparaison avec Strasbourg : l'autre ville branchée Vélo

In [ ]:
df_strasbourg_2023 = df_accidents_2023[df_accidents_2023['dep']=='67']
df_strasbourg_2022 = df_accidents_2022[df_accidents_2022['dep']=='67']

In [ ]:
df_strasbourg_2023 = pd.merge(df_strasbourg_2023,df_vehicules_2023, how='inner',on='Num_Acc')
df_strasbourg_2022 = pd.merge(df_strasbourg_2022,df_vehicules_2022, how='inner',on='Num_Acc')

In [ ]:
print(len(df_strasbourg_2023),len(df_strasbourg_2022))

100 121


## Affinage de l'analyse

## Fonction  (à affiner)

In [ ]:
def Nombre_accidents(fichier_vehicule,fichier_caracteristique,département):
    df_vehicule = pd.read_csv(fichier_vehicule,sep=';')
    df_accident = pd.read_csv(fichier_caracteristique,sep=';')
    df_accident=df_accident[df_accident['dep']==département]
    df_accident = pd.merge(df_accident,df_vehicule,how='inner',on='Num_Acc')
    return len(df_accident)

# Grenoble

In [ ]:
df_carrefour = pd.read_csv('amenagement_carrefour.csv',sep=',')
df_carrefour

,locali_id,geo_point_2d,carref_id,carref_datesupp,carref_datecre,carref_datemaj,accessi_desc,id
0,7,"45.1592442108994,5.73097602290016",250,NaN,2.000010e+13,NaN,Accessible,124
1,7,"45.1784747317108,5.72400680810226",255,NaN,2.000010e+13,NaN,Accessible,125
2,7,"45.1787899004849,5.72390867736167",254,NaN,2.000010e+13,NaN,Accessible,126
3,7,"45.1919814588737,5.72857005674412",302,NaN,2.000010e+13,NaN,Accessible,127
4,7,"45.187256332043,5.71100083692238",261,NaN,2.000010e+13,NaN,Accessible,128
...,...,...,...,...,...,...,...,...
1301,7,"45.1842581189286,5.74677811022395",1068,NaN,2.000010e+13,NaN,Pas de passage piéton,1283
1302,7,"45.1845794749506,5.7134104469574",1075,NaN,2.000010e+13,NaN,Pas de passage piéton,1284
1303,7,"45.1738004173595,5.74005400273794",1170,NaN,2.000010e+13,NaN,Pas de passage piéton,1285
1304,7,"45.1669120305242,5.73823523806341",1182,NaN,2.000010e+13,NaN,Pas de passage piéton,1286


In [ ]:
df_carrefour['latitude'] = [element[0] for element in df_carrefour['geo_point_2d'].apply(lambda x : x.split(','))]
df_carrefour['longitude'] = [element[1] for element in df_carrefour['geo_point_2d'].apply(lambda x : x.split(','))]



In [ ]:
df_carrefour['accessi_desc'].unique()

array(['Accessible', 'Inaccessible', 'Part. accessible',
       'Pas de passage piéton'], dtype=object)

In [ ]:
df_carrefour.groupby('accessi_desc')['carref_id'].count().reset_index()

,accessi_desc,carref_id
0,Accessible,774
1,Inaccessible,194
2,Part. accessible,116
3,Pas de passage piéton,222


On prend un point qui va nous servir de centre

In [ ]:
for index, row in df_carrefour[df_carrefour['accessi_desc']=='Accessible'].iterrows():
    folium.Marker([row['latitude'], row['longitude']],icon=folium.Icon(color="green")).add_to(map_accidents_carrefour_Grenoble)

for index, row in df_carrefour[df_carrefour['accessi_desc']=='Pas de passage piéton'].iterrows():
    folium.Marker([row['latitude'], row['longitude']],icon=folium.Icon(color="white")).add_to(map_accidents_carrefour_Grenoble)

for index, row in df_carrefour[df_carrefour['accessi_desc']=='Part. accessible'].iterrows():
    folium.Marker([row['latitude'], row['longitude']],icon=folium.Icon(color="orange")).add_to(map_accidents_carrefour_Grenoble)

for index, row in df_carrefour[df_carrefour['accessi_desc']=='Inaccessible'].iterrows():
    folium.Marker([row['latitude'], row['longitude']],icon=folium.Icon(color="red")).add_to(map_accidents_carrefour_Grenoble)

In [ ]:
map_accidents_carrefour_Grenoble.save('carte_carrefour_accidents.html')

<iframe src="ma_carte.html" width="800" height="600"></iframe>

Ce qu'on remarque c'est que sur 10 accidents dans le centre ville élargit de Grenoble , il y en a 4 ou 5 qui se sont passés très proche d'un carrefour.
Ce qu'il s'est passé, nous manquons d'informations sur les conditions de cet accident mais le risque que ce soit du à l'agencement ou à la disposition des véhicules à proximité du carrefour doit très certainement jouer.
L'accessibilité du carrefour ne semble pas être un facteur déterminant de cause de l'accident.
Les données gps relatives aux carrefours exterieurs à Grenoble manque et ne couvrent pas les accidents en dehors de la zone du centre et hypercentre.

## Deuxième indicateur : comptage permanent vélo

In [ ]:
import pandas as pd
"Chemin à reprendre si on utilise JupyterNotebook BaseDeDonnées/comptages_velos_permanents.csv"
df=pd.read_csv("BaseDeDonnées/comptages_velos_permanents.csv")
df_vf=df[df['territoire'] == 'Grenoble Alpes Metropole']

#print(df.info())
#print(df.head(3))
df_vf.head()

FileNotFoundError: [Errno 2] No such file or directory: 'BaseDeDonnées/comptages_velos_permanents.csv'

In [ ]:
# Afficher ou vérifier les premières lignes du nouveau DataFrame
colonnes=['nom_post', 'modes','localisation','type_compteur','type_axe','tmj_2019','tmj_2020','tmj_2021','tmj_2022']
colonnes_bis=['nom_post','localisation','type_axe','tmj_2019','tmj_2020','tmj_2021','tmj_2022']

print(df_vf[colonnes_bis].head(10))

#### Comme tous les compteurs n'ont pas été installés au même moment, il existe beaucoup de valeurs manquantes (NaN), il sera alors nécessaire de les ignorer au moment des analyses car il ne conviendrait pas de les remplacer par des zéros (fausseraient les résultats) de supprimer les lignes associées (trop nombreuses)
#### Ce qu'il faudrait c'est donc au moment de faire les analyses, passer outre ces valeurs vides (si l'on cherche a avoir des moyennes par années ou par ville par exemple)

## Troisième indicateur : Kilomètre piste cyclable

In [109]:
from shapely.geometry import Point
from shapely.geometry import LineString
import geopandas as gpd
from shapely.wkt import load

In [110]:
df_piste = pd.read_csv('pistes_cyclables.xls')

In [111]:
df_piste['lat'] = [element[0] for element in df_piste['geo_point_2d'].str.split(',')]
df_piste['long'] = [element[1] for element in df_piste['geo_point_2d'].str.split(',')]

In [112]:
df_piste['lat'] = df_piste['lat'].astype(object)
df_piste['long'] = df_piste['long'].astype(object)



In [113]:
df_piste['segment'] = df_piste['ogc_fid']

On prend notre point d'encrage de la map

In [114]:
map_piste_cyclable = folium.Map(location=[45.188529, 5.724524],
                                              zoom_start=14,
                                              titles = 'Map Piste Cyclable')

Transformation de geo_shape en json

In [115]:
df_piste['geo_shape'] = df_piste['geo_shape'].apply(lambda x: LineString(eval(x)['coordinates']))

In [116]:
df_piste['coordonnees']= df_piste['geo_shape'].apply(lambda x : list(x.coords)).to_list()


In [117]:
def reverse_coordonnees(liste):
  return [(coordonnees[1],coordonnees[0]) for coordonnees in liste ]

In [118]:
df_piste['coordonnees']= df_piste['coordonnees'].apply(lambda x: reverse_coordonnees(x))

In [119]:
for cords in df_piste['coordonnees']:
    folium.PolyLine(cords,color = 'red').add_to(map_piste_cyclable)

In [120]:
df_piste['lat'] = df_piste['lat'].astype(float)
df_piste['long'] = df_piste['long'].astype(object)

In [121]:
map_piste_cyclable #affichage de la carte

In [122]:
map_piste_cyclable.save('map_pistes_cyclables.html')

Explications

### Vérification des type LineString

## traffic routier

In [3]:
!pip install pyproj

In [4]:
import pyproj
from pyproj import Transformer

In [ ]:
from lxml import etree

root = etree.parse('qtvDir.xml')
for elem in root.iter():
    print( elem.text)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
                    

                        

                            
83.333336

            
None
2025-01-18T14:42:00.000+01:00

                

                    

                        

                            
2100

                

                    

                        

                            
82.0

            
None
2025-01-18T14:42:00.000+01:00

                

                    

                        

                            
930

                

                    

                        

                            
73.0

            
None
2025-01-18T14:42:00.000+01:00

                

                    

                        

                            
1840

                

                    

                        

                            
80.0

            
None
2025-01-18T14:42:00.000+01:00

                

               

In [ ]:
## Code communes Grenoble
L_communes = [38185,38421,38563,38239] #Grenoble , Saint martin d'heres , Voiron , Moirans

In [97]:
df_traffic = pd.read_csv('tmj_axes_routier.csv',sep=';')

In [98]:
#print(df_traffic.dtypes)
df_traffic = df_traffic[df_traffic['depPrD']==38]

In [99]:
df_traffic = df_traffic.dropna() # Il y en a très peu

In [36]:
df_traffic.columns

Index(['dateReferentiel', 'route', 'longueur', 'prD', 'depPrD',
       'concessionPrD', 'absD', 'cumulD', 'xD', 'yD', 'zD', 'prF', 'depPrF',
       'concessionPrF', 'absF', 'cumulF', 'xF', 'yF', 'zF',
       'anneeMesureTrafic', 'typeComptageTrafic', 'typeComptageTrafic_lib',
       'TMJA', 'ratio_PL'],
      dtype='object')

In [100]:
df_traffic= df_traffic.rename(columns ={'prD':'Point de repère routier départ'})
df_traffic = df_traffic.rename(columns ={'prF':'Point de repère routier fin' })
df_traffic = df_traffic.rename(columns ={'depPrD':'Département PR départ' })
df_traffic = df_traffic.rename(columns ={'depPrF':'Département PR fin ' })

In [101]:
df_traffic['xD']=df_traffic['xD'].str.replace(',','.')
df_traffic['xF']=df_traffic['xF'].str.replace(',','.')
df_traffic['yD']=df_traffic['yD'].str.replace(',','.')
df_traffic['yF']=df_traffic['yF'].str.replace(',','.')



df_traffic['xD']=df_traffic['xD'].astype(float)
df_traffic['yD']=df_traffic['yD'].astype(float)
df_traffic['xF']=df_traffic['xF'].astype(float)
df_traffic['yF']=df_traffic['yF'].astype(float)

### Partie pas concluante

In [24]:
def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[1], lonlat[0]

In [38]:


def xy_to_latlon_2(x, y, source_epsg, dest_epsg=4326):
    """Convertit des coordonnées X,Y dans un système de projection donné en latitude et longitude (WGS84).

    Args:
        x (float): Coordonnée X.
        y (float): Coordonnée Y.
        source_epsg (int): Code EPSG du système de projection source.
        dest_epsg (int, optional): Code EPSG du système de projection destination (WGS84 par défaut).

    Returns:
        tuple: Tuple contenant la latitude et la longitude.
    """

    transformer = Transformer.from_crs(source_epsg, dest_epsg)
    lon, lat = transformer.transform(x, y)
    return lat, lon

In [46]:
#df_traffic['coordonnees_gps_debut'] = [xy_to_lonlat(element_x,element_y) for element_x,element_y in zip(df_traffic['xD'],df_traffic['yD'])]
df_traffic['coordonnees_gps_debut_2'] = [xy_to_latlon_2(element_x,element_y,32632) for element_x,element_y in zip(df_traffic['xD'],df_traffic['yD'])]

In [44]:
df_traffic.route.values

array(['A0007', 'A0007', 'A0007', 'A0007', 'A0007N', 'A0007N', 'A0007N',
       'A0007N', 'A0041', 'A0041', 'A0041', 'A0041', 'A0041', 'A0041',
       'A0041', 'A0041', 'A0041', 'A0041', 'A0041', 'A0043', 'A0043',
       'A0043', 'A0043', 'A0043', 'A0043', 'A0043', 'A0043', 'A0043',
       'A0043', 'A0043', 'A0048', 'A0048', 'A0048', 'A0048', 'A0048',
       'A0048', 'A0049', 'A0049', 'A0049', 'A0049', 'A0049', 'A0051N',
       'A0051N', 'A0051N', 'A0051N', 'A0051N', 'A0432', 'A0432', 'A0432',
       'N0007', 'N0007', 'N0007', 'N0007', 'N0007', 'N0007', 'N0007',
       'N0007', 'N0007', 'N0007', 'N0007', 'N0007', 'N0007', 'N0007',
       'N0007', 'N0007', 'N0007', 'N0007', 'N0007', 'N0007', 'N0007',
       'N0007', 'N0007', 'N0007', 'N0007', 'N0007', 'N0007', 'N0007',
       'N0007', 'N0007', 'N0007', 'N0007', 'N0007', 'N0007', 'N0007',
       'N0007', 'N0007', 'N0007', 'N0007', 'N0007', 'N0007', 'N0007',
       'N0007', 'N0007', 'N0007', 'N0007', 'N0007', 'N0007', 'N0007',
       'N00

In [48]:
df_traffic['coordonnees_gps_fin'] = [xy_to_lonlat(element_x,element_y) for element_x,element_y in zip(df_traffic['xF'],df_traffic['yF'])]


In [54]:
df_traffic['coordonnees'] = [(element_d,element_f) for element_d,element_f in zip(df_traffic['coordonnees_gps_debut'],df_traffic['coordonnees_gps_fin'])]


In [57]:
df_traffic['coordonnees'] = df_traffic['coordonnees'].apply(lambda x : reverse_coordonnees(x))

In [58]:
map_route = folium.Map(location=[45.188529, 5.724524],
                                              zoom_start=14,
                                              titles = 'Map Piste Cyclable')

In [59]:
for cords in df_traffic['coordonnees']:
    folium.PolyLine(cords,color = 'blue').add_to(map_route)

In [60]:
#map_route

La partie de visualisation des axes routiers est compliquée car on a des coordonnées X,Y qui sont fausses , du moins qui ne correspondent pas à des coordonnées GPS une fois conversion faite à des coordonnées en France et encore moins à Grenoble .

### Fin de partie peu concluante

### Approche par le regroupement

In [93]:
import seaborn as sns
import matplotlib.pyplot as plt

In [61]:
df_traffic.columns

Index(['dateReferentiel', 'route', 'longueur',
       'Point de repère routier départ', 'Département PR départ',
       'concessionPrD', 'absD', 'cumulD', 'xD', 'yD', 'zD',
       'Point de repère routier fin', 'Département PR fin ', 'concessionPrF',
       'absF', 'cumulF', 'xF', 'yF', 'zF', 'anneeMesureTrafic',
       'typeComptageTrafic', 'typeComptageTrafic_lib', 'TMJA', 'ratio_PL',
       'coordonnees_gps_debut_2', 'coordonnees_gps_fin_2', 'coordonnees_2',
       'coordonnees'],
      dtype='object')

In [126]:
df_traffic.dtypes

,0
dateReferentiel,object
route,object
longueur,object
Point de repère routier départ,int64
Département PR départ,int64
concessionPrD,object
absD,object
cumulD,object
xD,float64
yD,float64


In [102]:
df_traffic['longueur']=df_traffic['longueur'].str.replace(',','.')
df_traffic['ratio_PL']=df_traffic['ratio_PL'].str.replace(',','.')
df_traffic['ratio_PL']=df_traffic['ratio_PL'].astype(float)
df_traffic['ratio_PL']=df_traffic['ratio_PL']/100

In [103]:
df_traffic['Nb_poids_lourds'] = round(df_traffic['TMJA']* df_traffic['ratio_PL'])

In [104]:
df_axes = df_traffic.groupby('route')[['longueur','TMJA','Nb_poids_lourds']].sum().reset_index().sort_values(by='TMJA',ascending=False)

df_axes['ratio_PL']=round((df_axes['Nb_poids_lourds']/df_axes['TMJA'])*100)

df_axes

,route,longueur,TMJA,Nb_poids_lourds,ratio_PL
8,N0007,"503021124415163472867194399791299342,3706,7648...",1175847.0,904151.0,77.0
10,N0087,"5001988745174619915457,8588,23505",733132.0,377954.0,52.0
3,A0043,125545606245620542006165311711348387065012755,682461.0,73276.0,11.0
2,A0041,5001450780301028203540148010175944539502849,509799.0,20175.0,4.0
1,A0007N,90646369184704,378864.0,60368.0,16.0
4,A0048,2601590454130380820174935,312627.0,19724.0,6.0
0,A0007,102457385265028826,291786.0,56487.0,19.0
9,N0085,1988175124490538311,211442.0,156464.0,74.0
5,A0049,95121342710488110731865,118883.0,8202.0,7.0
6,A0051N,62229431435142326843,80092.0,2894.0,4.0


La route avec le plus de traffic par jour sur un an est la route nationale 7 qui est du côté de Valence qui ne rentre donc pas dans notre étude , suivi de la Route Nationale 87 avec 733 132 véhicules en moyen par jour sur l'agglomération Grenobloise, route plus connue sous le nom de 'Rocade Sud'

A43 : ne rentre pas dans l'étude

A41 : Qui vient de Chambéry
Tallonnée par A480 qui traverse Grenoble du nord au sud en venant de Lyon

Si on reprend notre carte des pistes de vélo :

On voit que les pistes cyclables sont tout autour de cette Rocade Sud mais c'est problèmatique , on ne peut pas la longer , aller tout droit comme le ferait la piste cyclable qui longe l'A48

Cela peut se conçevoir du fait des travaux autour qui durent depuis plus années.

On l'a vu , un des critères à l'utilisation des pistes cyclables c'est quelles soient directes , avec le moins de détours possibles.

Ici , quelqu'un qui veut se rendre depuis Meylan à Échirolles risque de prendre inévitablement la voiture.

Murianettes - Gières - Poisat - Eybens = > sinon faire une piste cyclable plus adaptée sur cette portion , comme on l'a vu , pour avoir une piste cyclable beaucoup plus adaptée.

#### Passage en csv pour streamLit

In [124]:
df_axes = df_axes.to_csv('Axes_routiers.csv')

In [125]:
df_axes_test = pd.read_csv('Axes_routiers.csv')
df_axes_test

,Unnamed: 0,route,longueur,TMJA,Nb_poids_lourds,ratio_PL
0,8,N0007,"503021124415163472867194399791299342,3706,7648...",1175847.0,904151.0,77.0
1,10,N0087,"5001988745174619915457,8588,23505",733132.0,377954.0,52.0
2,3,A0043,125545606245620542006165311711348387065012755,682461.0,73276.0,11.0
3,2,A0041,5001450780301028203540148010175944539502849,509799.0,20175.0,4.0
4,1,A0007N,90646369184704,378864.0,60368.0,16.0
5,4,A0048,2601590454130380820174935,312627.0,19724.0,6.0
6,0,A0007,102457385265028826,291786.0,56487.0,19.0
7,9,N0085,1988175124490538311,211442.0,156464.0,74.0
8,5,A0049,95121342710488110731865,118883.0,8202.0,7.0
9,6,A0051N,62229431435142326843,80092.0,2894.0,4.0


In [ ]:
# Fonction pour vérifier le type de géométrie
def is_linestring(geo_shape):
    try:
        shape = json.loads(geo_shape)
        return shape['type'] == 'LineString'
    except:
        return False

# Vérifier si toutes les géométries sont des LineString
df['is_linestring'] = df['geo_shape'].apply(is_linestring)

# Résumé des résultats
print("Résumé des types de géométries :")
print(df['is_linestring'].value_counts())

# Identifier les géométries non conformes, s'il y en a
non_linestring_rows = df[~df['is_linestring']]
if not non_linestring_rows.empty:
    print("Lignes avec des géométries non conformes :")
    print(non_linestring_rows)
else:
    print("Toutes les géométries sont des LineString.")

Résumé des types de géométries :
is_linestring
True    1202
Name: count, dtype: int64
Toutes les géométries sont des LineString.


### Calcul des distances de la piste cyclable

In [ ]:
# Fonction pour calculer la distance d'une piste cyclable à partir de la colonne geo_shape
def calculate_distance(geo_shape):
    try:
        # Charger les coordonnées GeoJSON
        shape = json.loads(geo_shape)
        if shape['type'] == 'LineString':
            coordinates = shape['coordinates']
            # Calculer la distance totale entre les points consécutifs
            distances = [
                geodesic(coordinates[i], coordinates[i+1]).kilometers
                for i in range(len(coordinates) - 1)
            ]
            return sum(distances)
    except:
        return None  # Retourner None si un problème survient

# Ajouter une colonne pour les distances
df['distance_km'] = df['geo_shape'].apply(calculate_distance)

# Enregistrer le DataFrame avec la nouvelle colonne dans un nouveau fichier CSV
df.to_csv('fichier_avec_distances.csv', index=False)

# Afficher les premières lignes pour vérifier
print(df.head())

## Streamlit

voir fichier streamlit_script.py